In [4]:
import cvxpy as cp
import numpy as np
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt

np.random.seed(42)
torch.manual_seed(42)

warnings.filterwarnings("ignore")
InteractiveShell.ast_node_interactivity = "all"
sys.path.insert(1,'E:\\User\\Stevens\\Spring 2024\\PTO - Fairness\\myGit\\myUtils')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

num_data = 100
num_features = 4
num_items = 3

cuda


In [5]:
def genData(num_data, num_features, num_items, seed=42, Q=100, dim=1, deg=1, noise_width=0.5, epsilon=0.1):
    rnd = np.random.RandomState(seed)
    n = num_data
    p = num_features
    m = num_items
    
    x = rnd.normal(0, 1, (n, m, p))
    B = rnd.binomial(1, 0.5, (m, p))

    c = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            values = (np.dot(B[j], x[i, j].reshape(p, 1)).T / np.sqrt(p) + 3) ** deg + 1
            values *= 5
            values /= 3.5 ** deg
            epislon = rnd.uniform(1 - noise_width, 1 + noise_width, 1)
            values *= epislon
            values = np.ceil(values)
            c[i, j] = values

    c = c.astype(np.float64)
    r = rnd.normal(0, 1, (n, m))
    r = 1 / (1 + np.exp(-r))

    return x, r, c, Q

In [6]:
class optModel:
    def __init__(self, x, r, c, Q, alpha):
        self.alpha = alpha
        self.Q = Q
        self.r = r
        self.c = c
        self.x = x
        self.num_data, self.num_items, self.num_features = x.shape

    def setObj(self, r, c):
        if self.alpha == 1:
            self.objective = cp.sum(cp.log(cp.multiply(r, self.d)))
        else:
            self.objective = cp.sum(cp.power(cp.multiply(r, self.d), 1 - self.alpha)) / (1 - self.alpha)
        
        self.constraints = [
            self.d >= 0,
            cp.sum(cp.multiply(c, self.d)) <= self.Q
        ]
        self.problem = cp.Problem(cp.Maximize(self.objective), self.constraints)

    def solve(self, closed=False):
        opt_sol = []
        opt_val = []
        if closed:
            return self.solveC()

        for i in range(self.num_data):
            self.d = cp.Variable(self.num_items)
            self.setObj(self.r[i], self.c[i])
            self.problem.solve(abstol=1e-9, reltol=1e-9, feastol=1e-9)
            opt_sol.append(self.d.value.reshape(1, self.num_items))
            opt_val.append(self.problem.value)

        opt_sol = np.concatenate(opt_sol)
        return opt_sol, opt_val

    def solveC(self):
        if self.alpha == 1:
            return "Work in progress"
        
        opt_sols_c = []
        opt_vals_c = []
        for i in range(self.num_data):
            S = np.sum(self.c[i] ** (1 - 1 / self.alpha) * self.r[i] ** (-1 + 1 / self.alpha))
            opt_sol_c = (self.c[i] ** (-1 / self.alpha) * self.r[i] ** (-1 + 1 / self.alpha) * self.Q) / S
            opt_val_c = np.sum((self.r[i] * opt_sol_c) ** (1 - self.alpha)) / (1 - self.alpha)
            opt_sols_c.append(opt_sol_c)
            opt_vals_c.append(opt_val_c)
        
        opt_sols_c = np.array(opt_sols_c)
        return opt_sols_c, opt_vals_c

In [7]:
# Test optModel with synthetic data
x, r, c, Q = genData(10, 4, 3)
alpha = 0.5

# Create an instance of the optModel class
optmodel = optModel(x, r, c, Q, alpha)

# Solve the optimization problem
opt_sol, opt_val = optmodel.solve()

print("Optimal solution:", opt_sol[0])
print("Objective value:", opt_val[0])

opt_sol_c, opt_val_c = optmodel.solve(closed=True)

print("Optimal solution (closed form):", opt_sol_c[0])
print("Objective value (closed form):", opt_val_c[0])

# Are they the same?
np.allclose(opt_sol, opt_sol_c, atol=1e-4, rtol=1e-4)
np.allclose(opt_val, opt_val_c)

Optimal solution: [ 1.89716597 12.58886077  5.74453819]
Objective value: 10.525035953903501
Optimal solution (closed form): [ 1.89715409 12.58850965  5.74478812]
Objective value (closed form): 10.525035956707143


True

True